In [1]:
%load_ext autoreload
%load_ext autotime
%load_ext nb_black
%autoreload 2

time: 3.33 s (started: 2023-08-04 23:14:52 +03:00)


<IPython.core.display.Javascript object>

time: 2.53 s (started: 2023-08-04 23:14:56 +03:00)


<IPython.core.display.Javascript object>

In [3]:
from bamt.networks.hybrid_bn import HybridBN
import bamt.preprocessors as pp

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from pgmpy.estimators import K2Score

C:\ProgramData\Anaconda3\lib\site-packages\bamt\log.py:20: UserWarning: Reading log path location from config file failed. Default location will be used instead.
  warnings.warn(


time: 14.1 s (started: 2023-08-04 23:14:58 +03:00)


<IPython.core.display.Javascript object>

In [23]:
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from sklearn.metrics import classification_report
from sklearn.dummy import DummyClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.metrics import f1_score

time: 0 ns (started: 2023-08-04 23:50:44 +03:00)


<IPython.core.display.Javascript object>

In [5]:
pd.set_option("display.precision", 5)
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)

time: 0 ns (started: 2023-08-04 23:15:13 +03:00)


<IPython.core.display.Javascript object>

In [11]:
data = pd.read_csv(r"real_data\vk_data.csv")
data
# data = data.sample(10000)
data = data.iloc[:, int(data.shape[1] / 2) : -1]

time: 312 ms (started: 2023-08-04 23:16:12 +03:00)


<IPython.core.display.Javascript object>

In [12]:
cat_features = [
    "sex",
    "is_parent",
    "relation",
    "is_driver",
]
cat_features = data.select_dtypes("object").columns.values.tolist()


if cat_features:
    data[cat_features] = data[cat_features].astype(str)

target = "has_pets"
X = data
# X = data[["age", "sex", "has_pets", "is_parent", "relation", "is_driver", "tr_per_month", "median_tr", "mean_tr"]]

time: 16 ms (started: 2023-08-04 23:16:15 +03:00)


<IPython.core.display.Javascript object>

In [13]:
X[target].value_counts(normalize=True)

0    0.94807
1    0.05193
Name: has_pets, dtype: float64

time: 31 ms (started: 2023-08-04 23:16:15 +03:00)


<IPython.core.display.Javascript object>

In [14]:
X_train, X_test = train_test_split(X, test_size=5000, shuffle=True, stratify=X[target])

time: 47 ms (started: 2023-08-04 23:16:16 +03:00)


<IPython.core.display.Javascript object>

In [15]:
X_train[target].value_counts(normalize=True)

0    0.94808
1    0.05192
Name: has_pets, dtype: float64

time: 0 ns (started: 2023-08-04 23:16:18 +03:00)


<IPython.core.display.Javascript object>

In [16]:
X_test[target].value_counts(normalize=True)

0    0.948
1    0.052
Name: has_pets, dtype: float64

time: 15 ms (started: 2023-08-04 23:16:18 +03:00)


<IPython.core.display.Javascript object>

In [24]:
gbm = CatBoostClassifier(cat_features=cat_features, n_estimators=100)  # , eval_fraction=0.5
gbm.fit(X_train.drop(columns=[target]), X_train[target], plot=False, verbose=False)
preds = gbm.predict(X_test.drop(columns=[target]))
print(f"Catboost: f1={f1_score(y_true=X_test[target],y_pred=preds)}")
print(classification_report(y_true=X_test[target], y_pred=preds, digits=4))

Catboost: f1=0.952561669829222
              precision    recall  f1-score   support

           0     0.9981    0.9966    0.9974      4740
           1     0.9401    0.9654    0.9526       260

    accuracy                         0.9950      5000
   macro avg     0.9691    0.9810    0.9750      5000
weighted avg     0.9951    0.9950    0.9950      5000

time: 4.73 s (started: 2023-08-04 18:07:22 +03:00)


<IPython.core.display.Javascript object>

In [60]:
res = cross_validate(
    CatBoostClassifier(cat_features=cat_features, n_estimators=100, verbose=False),
    X.drop(columns=[target]),
    X[target],
    cv=StratifiedKFold(n_splits=100),
    scoring="f1",
)
print(f"{res['test_score'].mean()} +- {res['test_score'].std()}")

0.003529411764705882 +- 0.020069084834390568
time: 4min 46s (started: 2023-08-04 17:44:28 +03:00)


<IPython.core.display.Javascript object>

In [25]:
gbm = XGBClassifier(n_estimators=100, enable_categorical=True, tree_method="hist")
X_train[cat_features] = X_train[cat_features].astype("category")
X_test[cat_features] = X_test[cat_features].astype("category")
gbm.fit(X_train.drop(columns=[target]), X_train[target], verbose=False)
preds = gbm.predict(X_test.drop(columns=[target]))
print(f"XGBClassifier: f1={f1_score(y_true=X_test[target],y_pred=preds)}")
print(classification_report(y_true=X_test[target], y_pred=preds, digits=4))

XGBClassifier: f1=0.9581749049429658
              precision    recall  f1-score   support

           0     0.9983    0.9970    0.9977      4740
           1     0.9474    0.9692    0.9582       260

    accuracy                         0.9956      5000
   macro avg     0.9728    0.9831    0.9779      5000
weighted avg     0.9957    0.9956    0.9956      5000

time: 1.06 s (started: 2023-08-04 18:07:53 +03:00)


<IPython.core.display.Javascript object>

In [78]:
X[cat_features] = X[cat_features].astype("category")
res = cross_validate(
    XGBClassifier(n_estimators=100, enable_categorical=True, tree_method="hist"),
    X.drop(columns=[target]),
    X[target],
    cv=StratifiedKFold(n_splits=100),
    scoring="f1",
)
print(f"{res['test_score'].mean()} +- {res['test_score'].std()}")

0.01165032679738562 +- 0.038355493012708616
time: 36.3 s (started: 2023-08-04 17:56:35 +03:00)


<IPython.core.display.Javascript object>

In [61]:
res = cross_validate(DummyClassifier(strategy="uniform"), X.drop(columns=[target]), X[target], cv=StratifiedKFold(n_splits=10), scoring="f1")
print(f"{res['test_score'].mean()} +- {res['test_score'].std()}")

0.09801016083159965 +- 0.004471548010181664
time: 78 ms (started: 2023-08-04 17:49:15 +03:00)


<IPython.core.display.Javascript object>

In [68]:
for strategy in "uniform stratified most_frequent prior".split():
    dummy = DummyClassifier(strategy=strategy)
    dummy.fit(X_train.drop(columns=[target]), X_train[target])
    preds = dummy.predict(X_test.drop(columns=[target]))
    print(f"Dummy[{strategy}]: f1={f1_score(y_true=X_test[target],y_pred=preds)}")
    print(classification_report(y_true=X_test[target], y_pred=preds, digits=4))

Dummy[uniform]: f1=0.09411764705882353
              precision    recall  f1-score   support

           0     0.9480    0.5080    0.6615      4740
           1     0.0520    0.4923    0.0941       260

    accuracy                         0.5072      5000
   macro avg     0.5000    0.5002    0.3778      5000
weighted avg     0.9014    0.5072    0.6320      5000

Dummy[stratified]: f1=0.04098360655737705
              precision    recall  f1-score   support

           0     0.9476    0.9540    0.9508      4740
           1     0.0439    0.0385    0.0410       260

    accuracy                         0.9064      5000
   macro avg     0.4957    0.4962    0.4959      5000
weighted avg     0.9006    0.9064    0.9035      5000

Dummy[most_frequent]: f1=0.0
              precision    recall  f1-score   support

           0     0.9480    1.0000    0.9733      4740
           1     0.0000    0.0000    0.0000       260

    accuracy                         0.9480      5000
   macro avg     0

<IPython.core.display.Javascript object>

In [17]:
X_train[cat_features] = X_train[cat_features].astype("str")
X_test[cat_features] = X_test[cat_features].astype("str")
del data

time: 0 ns (started: 2023-08-04 23:16:25 +03:00)


<IPython.core.display.Javascript object>

In [18]:
encoder = preprocessing.LabelEncoder()
discretizer = preprocessing.KBinsDiscretizer(n_bins=5, encode="ordinal", strategy="quantile")

p = pp.Preprocessor([("encoder", encoder), ("discretizer", discretizer)])
discretized_data, est = p.apply(X_train)

time: 125 ms (started: 2023-08-04 23:16:26 +03:00)


<IPython.core.display.Javascript object>

In [19]:
bn = HybridBN(has_logit=True, use_mixture=True)  # init BN
info = p.info
info

{'types': {'top3': 'disc_num',
  'top3_mean': 'cont',
  'game_sum': 'cont',
  'is_gamer': 'disc_num',
  'parent_sum': 'cont',
  'is_parent': 'disc_num',
  'driver_sum': 'cont',
  'is_driver': 'disc_num',
  'pets_sum': 'cont',
  'has_pets': 'disc_num',
  'dress, fashion, color, style': 'cont',
  'massage, meditation, yoga': 'cont',
  'game, team, competition, sport': 'cont',
  'color, flowers, interior, design': 'cont',
  'iphone, apple, technology, equipment, smartphone': 'cont',
  'client, project, internet, business, company': 'cont',
  'war, history, army, politics': 'cont',
  'beauty, hair, manicure, salon': 'cont',
  'handmade, materials, products': 'cont',
  'work, schedule, earnings, money': 'cont',
  'music, concerts, rock, tickets, festivals': 'cont',
  'treatment, doctors, tests, clinic, health': 'cont',
  'school, education, question, topic': 'cont',
  'skin, face, care, cosmitology': 'cont',
  'book, reading, language, word, literature': 'cont',
  'participation, festival, 

time: 0 ns (started: 2023-08-04 23:16:26 +03:00)


<IPython.core.display.Javascript object>

In [20]:
bn.add_nodes(info)
bn.add_edges(discretized_data, scoring_function=("K2", K2Score))

  0%|          | 0/1000000 [00:00<?, ?it/s]

time: 50.7 s (started: 2023-08-04 23:16:27 +03:00)


<IPython.core.display.Javascript object>

In [21]:
del discretized_data
bn.get_info()

,name,node_type,data_type,parents,parents_types
0,game_sum,MixtureGaussian,cont,[],[]
1,driver_sum,MixtureGaussian,cont,[],[]
2,has_pets,Discrete,disc_num,[],[]
3,pets_sum,ConditionalMixtureGaussian,cont,[has_pets],[disc_num]
4,top3,Logit (LogisticRegression),disc_num,[pets_sum],[cont]
5,is_gamer,ConditionalLogit (LogisticRegression),disc_num,"[pets_sum, top3]","[cont, disc_num]"
6,is_parent,ConditionalLogit (LogisticRegression),disc_num,"[pets_sum, top3, is_gamer]","[cont, disc_num, disc_num]"
7,parent_sum,ConditionalMixtureGaussian,cont,"[top3, is_parent]","[disc_num, disc_num]"
8,top3_mean,ConditionalMixtureGaussian,cont,"[parent_sum, top3]","[cont, disc_num]"
9,"color, flowers, interior, design",ConditionalMixtureGaussian,cont,"[parent_sum, pets_sum, is_gamer, is_parent]","[cont, cont, disc_num, disc_num]"


time: 47 ms (started: 2023-08-04 23:17:22 +03:00)


<IPython.core.display.Javascript object>

In [22]:
bn.fit_parameters(X_train)

MemoryError: Unable to allocate 2.33 GiB for an array with shape (312487500,) and data type float64

time: 2min 52s (started: 2023-08-04 23:17:27 +03:00)


<IPython.core.display.Javascript object>

In [ ]:
pred = bn.predict(X_test.drop(columns=[target]), 5)

In [ ]:
preds = pred[target]
print(f"Bamt: f1={f1_score(y_true=X_test[target].astype(str),y_pred=preds,pos_label='1')}")
print(classification_report(y_true=X_test[target].astype(str), y_pred=preds, digits=4))